In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from glob import glob
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [52]:
adidas_train = glob('../input/nike-adidas-shoes-for-image-classification-dataset/train/adidas/*.jpg')
nike_train = glob('../input/nike-adidas-shoes-for-image-classification-dataset/train/nike/*.jpg')
adidas_val = glob('../input/nike-adidas-shoes-for-image-classification-dataset/validation/adidas/*.jpg')
nike_val = glob('../input/nike-adidas-shoes-for-image-classification-dataset/validation/nike/*.jpg')
adidas_test = glob('../input/nike-adidas-shoes-for-image-classification-dataset/test/adidas/*.jpg')
nike_test = glob('../input/nike-adidas-shoes-for-image-classification-dataset/test/nike/*.jpg')

In [53]:
len(adidas_train), len(nike_train), len(adidas_val), len(nike_val), len(adidas_test), len(nike_test)

## Let's see the examples of images

In [54]:
numbers = list(np.random.randint(27, size = 3))

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for i, number in enumerate(numbers):
    curr_img = plt.imread(adidas_train[number])
    axs[i].imshow(curr_img)
    plt.suptitle('Examples of adidas_train')
    
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for i, number in enumerate(numbers):
    curr_img = plt.imread(nike_train[number])
    axs[i].imshow(curr_img)
    plt.suptitle('Examples of nike_train')


## Different images have different shapes

## Let's preprocess our data

In [58]:
datagen = ImageDataGenerator(rescale=1./255)


In [59]:
train_generator = datagen.flow_from_directory('/kaggle/input/nike-adidas-shoes-for-image-classification-dataset/train/', target_size=(150, 150), class_mode='binary', batch_size = 20)
val_generator = datagen.flow_from_directory('/kaggle/input/nike-adidas-shoes-for-image-classification-dataset/validation/', target_size=(150, 150), class_mode='binary', batch_size = 20)
test_generator= datagen.flow_from_directory('/kaggle/input/nike-adidas-shoes-for-image-classification-dataset/test/', target_size=(150, 150), class_mode='binary', batch_size = 20)


In [60]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
 input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [61]:
model.summary()

In [62]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
 optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
 metrics='sparse_categorical_accuracy')



In [63]:
history = model.fit(train_generator, epochs=30, batch_size=64, validation_data = val_generator)

In [64]:
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.scatter(epochs, acc, color ='blue', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.scatter(epochs, loss, color='blue', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Due to small amount of images the overfitting problem is observed. The common way to solve this problem is to use data augmentation:

In [109]:
datagen = ImageDataGenerator(rescale=1./255, rotation_range = 25, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory('/kaggle/input/nike-adidas-shoes-for-image-classification-dataset/train/', target_size=(150, 150), class_mode='binary', batch_size = 64, shuffle=True, subset='training')
val_generator = datagen.flow_from_directory('/kaggle/input/nike-adidas-shoes-for-image-classification-dataset/validation/', target_size=(150, 150), class_mode='binary', batch_size = 64, shuffle=True)
test_generator= test_datagen.flow_from_directory('/kaggle/input/nike-adidas-shoes-for-image-classification-dataset/test/', target_size=(150, 150), class_mode='binary', shuffle=True)


In [110]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
 input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
 optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
 metrics='sparse_categorical_accuracy')



In [112]:
history = model.fit(train_generator, epochs = 100, validation_data = val_generator)

In [113]:
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.scatter(epochs, acc, color ='blue', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.scatter(epochs, loss, color='blue', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [114]:
history_aug = history

## Let's add batch normalization to our NN

In [115]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
 input_shape=(150, 150, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
 optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
 metrics='sparse_categorical_accuracy')


In [116]:
history_batch = model.fit(train_generator, epochs = 100, validation_data = val_generator)

In [117]:
acc = history_batch.history['sparse_categorical_accuracy']
val_acc = history_batch.history['val_sparse_categorical_accuracy']
loss = history_batch.history['loss']
val_loss = history_batch.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.scatter(epochs, acc, color ='blue', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.scatter(epochs, loss, color='blue', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [121]:
acc_aug = history_aug.history['val_sparse_categorical_accuracy']
loss_aug = history_aug.history['val_loss']
acc_batch = history_batch.history['val_sparse_categorical_accuracy']
loss_batch = history_batch.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc_aug, color ='blue', label='No BatchNormalization')
plt.plot(epochs, acc_batch, color ='green', label='With BatchNormalization')
plt.title('Accuracy depending on BatchNormalization layer')
plt.legend()
plt.figure()
plt.plot(epochs, loss_aug, color ='blue', label='No BatchNormalization')
plt.plot(epochs, loss_batch, color='green', label='With BatchNormalization')
plt.title('Loss depending on BatchNormalization layer')
plt.legend()



In [122]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'The total accuracy is {test_acc}')